In [3]:
using NeuralPDE, Lux, ModelingToolkit, Optimization,OptimizationOptimJL, OptimizationOptimisers
import ModelingToolkit: Interval, infimum, supremum


In [13]:
@parameters x y
@variables u(..)
Dxx = Differential(x)^2
Dyy = Differential(y)^2
Dy=Differential(y)


a=-1.0
b=0.0
g=1.0
k=2.0

# 2D Klein-Gordon Equation
eq = a*Dxx(u(x, y)) + Dyy(u(x, y))+ b*u(x,y)+ g*u(x,y)^k + x*cos(y) - x^2  *  cos(y)^2 ~0

# Boundary conditions (Dirichlet)
bcs = [u(-1, y) ~ -cos(y),
       u(1,y)~cos(y),
       u(x,0)~x,
       Dy(u(x,0))~0
]




# Spatial domain
domains = [x ∈ Interval(-1.0, 1.0),
           y ∈ Interval(0.0, 10.0)]

# Neural network
dim = 2  # number of dimensions (x, y)
chain = Lux.Chain(Dense(dim, 16, Lux.tanh), Dense(16, 16, Lux.tanh), Dense(16, 1))

discretization = PhysicsInformedNN(chain, QuadratureTraining())

@named pde_system = PDESystem(eq, bcs, domains, [x, y], [u(x, y)])
prob = discretize(pde_system, discretization)

callback = function (p, l)
    println("Current loss is: $l")
    return false
end

# Solve using ADAM optimizer
res = Optimization.solve(prob, BFGS(); callback = callback, maxiters = 100)

# Fine-tune the solution with a lower learning rate
prob = remake(prob, u0 = res.minimizer)
res = Optimization.solve(prob, BFGS(); callback = callback, maxiters = 80)

# Extract the solution
phi = discretization.phi


Current loss is: 2.6116543909664176
Current loss is: 1.9529288175371102
Current loss is: 1.7235765620042658
Current loss is: 1.298452407270856
Current loss is: 1.2499924074040065
Current loss is: 1.1608889052171583
Current loss is: 1.1033329419895133
Current loss is: 1.046051518458349
Current loss is: 1.0264848417426857
Current loss is: 0.9979852161084902
Current loss is: 0.9742121384702017
Current loss is: 0.9513413235921345
Current loss is: 0.9398413763069883
Current loss is: 0.9229626422517745
Current loss is: 0.9074313928636761
Current loss is: 0.8949498110744403
Current loss is: 0.8722517694461565
Current loss is: 0.8619202192426887
Current loss is: 0.854000888767487
Current loss is: 0.8317256780810494
Current loss is: 0.8248859421988781
Current loss is: 0.8082662659987188
Current loss is: 0.7875264144806253
Current loss is: 0.7812519447022661
Current loss is: 0.7728215588506672
Current loss is: 0.7629897818295115
Current loss is: 0.7539436079793393
Current loss is: 0.727604662944

NeuralPDE.Phi{Chain{@NamedTuple{layer_1::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_2::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_3::Dense{true, typeof(identity), typeof(glorot_uniform), typeof(zeros32)}}, Nothing}, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}}}(Chain{@NamedTuple{layer_1::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_2::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_3::Dense{true, typeof(identity), typeof(glorot_uniform), typeof(zeros32)}}, Nothing}((layer_1 = Dense(2 => 16, tanh_fast), layer_2 = Dense(16 => 16, tanh_fast), layer_3 = Dense(16 => 1)), nothing), (layer_1 = NamedTuple(), layer_2 = NamedTuple(), layer_3 = NamedTuple()))

In [26]:
print(phi([0.4,-0.3]))

LoadError: MethodError: no method matching (::NeuralPDE.Phi{Chain{@NamedTuple{layer_1::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_2::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_3::Dense{true, typeof(identity), typeof(glorot_uniform), typeof(zeros32)}}, Nothing}, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}}})(::Vector{Float64})

[0mClosest candidates are:
[0m  (::NeuralPDE.Phi{<:LuxCore.AbstractExplicitLayer})(::AbstractArray, [91m::Any[39m)
[0m[90m   @[39m [36mNeuralPDE[39m [90mC:\Users\wavef\.julia\packages\NeuralPDE\IUP3N\src\[39m[90m[4mpinn_types.jl:513[24m[39m
[0m  (::NeuralPDE.Phi{<:LuxCore.AbstractExplicitLayer})([91m::Number[39m, [91m::Any[39m)
[0m[90m   @[39m [36mNeuralPDE[39m [90mC:\Users\wavef\.julia\packages\NeuralPDE\IUP3N\src\[39m[90m[4mpinn_types.jl:507[24m[39m


In [20]:
using Statistics


LoadError: LoadError: UndefVarError: `@sprintf` not defined
in expression starting at In[46]:26

In [ ]:
using Plots

# Set the step size for the grid
dx = 0.01f0

# Create the grid for the domain
domains = [x ∈ Interval(-1.0, 1.0),
           y ∈ Interval(0.0, 1.0)]



xs, ys = [infimum(d.domain):dx:supremum(d.domain) for d in domains]

# Define the analytic solution function
analytic_sol_func(x, y) = x*cos(y)

# Predict the solution using the trained model
u_predict = reshape([first(phi([x, y], res.minimizer)) for x in xs for y in ys],
    (length(xs), length(ys)))

# Calculate the analytic solution on the grid
println(u_predict)

u_real = reshape([analytic_sol_func(x, y) for x in xs for y in ys],
    (length(xs), length(ys)))
println(u_real)
# Calculate the difference between the predicted and real solution
diff_u = abs.(u_predict .- u_real)

# Calculate the mean squared error
#mse = mean((u_predict .- u_real).^2)

#println("Mean Squared Error (MSE): ", mse)

# Plot the results
p1 = plot(xs, ys, u_real, linetype = :contourf, title = "Analytic Solution")
p2 = plot(xs, ys, u_predict, linetype = :contourf, title = "Predicted Solution")
p3 = plot(xs, ys, diff_u, linetype = :contourf, title = "Error ")

# Display all plots together
plot(p1, p2, p3, layout = (1, 3), size=(1200, 400))



In [45]:
println(u_predict)

[-0.9994601855602222 -0.540964147715392 -0.5381445373233958 -0.5349085939460713 -0.531260750148742 -0.5272055424794385 -0.5227476063894924 -0.5178916709652838 -0.5126425534743492 -0.5070051537303543 -0.5009844482827038 -0.49458548443778494 -0.48781337412050496 -0.48067328758612293 -0.4731704469942172 -0.4653101198582593 -0.4570976123860496 -0.44853826272828823 -0.43963743415431644 -0.43040050817614495 -0.4208328776437914 -0.4109399398369632 -0.40072708958007863 -0.3901997124095017 -0.37936317782371887 -0.3682228326488862 -0.3567839945539094 -0.34505194575049625 -0.3330319269150964 -0.32072913137068193 -0.3081486995670908 -0.2952957138994178 -0.28217519390423884 -0.2687920918734167 -0.2551512889251022 -0.24125759157090076 -0.2271157288171879 -0.21273034983737416 -0.19810602225021157 -0.1832472310371811 -0.16815837812972345 -0.15284378269425747 -0.13730768213978972 -0.12155423386958342 -0.10558751779442122 -0.08941153962098913 -0.07303023492440841 -0.05644747400924144 -0.0396670675584219

In [40]:
@time u_predict = reshape([first(phi([x, y], res.minimizer)) for x in xs for y in ys],
                          (length(xs), length(ys)))

@time u_real = reshape([analytic_sol_func(x, y) for x in xs for y in ys],
                       (length(xs), length(ys)))


  9.159102 seconds (62.18 M allocations: 3.381 GiB, 18.34% gc time, 3.08% compilation time)
  1.536953 seconds (8.11 M allocations: 513.838 MiB, 17.71% gc time, 27.39% compilation time)


2001×1001 Matrix{Float64}:
 1.0       0.541144  0.543661  0.546171  …  0.999996  0.999998  1.0
 1.0       1.0       0.542824  0.545337     0.999992  0.999996  0.999998
 0.999998  1.0       1.0       0.544503     0.999988  0.999992  0.999996
 0.999996  0.999998  1.0       1.0          0.999982  0.999988  0.999992
 0.999992  0.999996  0.999998  1.0          0.999976  0.999982  0.999988
 0.999988  0.999992  0.999996  0.999998  …  0.999968  0.999976  0.999982
 0.999982  0.999988  0.999992  0.999996     0.99996   0.999968  0.999976
 0.999976  0.999982  0.999988  0.999992     0.99995   0.99996   0.999968
 0.999968  0.999976  0.999982  0.999988     0.99994   0.99995   0.99996
 0.99996   0.999968  0.999976  0.999982     0.999929  0.99994   0.99995
 0.99995   0.99996   0.999968  0.999976  …  0.999916  0.999928  0.99994
 0.99994   0.99995   0.99996   0.999968     0.999903  0.999916  0.999928
 0.999928  0.99994   0.99995   0.99996      0.999889  0.999903  0.999916
 ⋮                              

In [ ]:
chain = Lux.Chain(Dense(2, 16, Lux.σ), Dense(16, 16, Lux.σ), Dense(16, 1))
discretization = PhysicsInformedNN(chain, GridTraining(dx))

@named pde_system = PDESystem(eq, bcs, domains, [t, x], [u(t, x)])
prob = discretize(pde_system, discretization)

callback = function (p, l)
    println("Current loss is: $l")
    return false
end


res = Optimization.solve(prob, ADAM(0.1); callback = callback, maxiters = 100)
prob = remake(prob, u0 = res.minimizer)
res = Optimization.solve(prob, ADAM(0.01); callback = callback, maxiters = 80)
phi = discretization.phi

In [28]:
using Plots

ts, xs = [infimum(d.domain):dx:supremum(d.domain) for d in domains]
function analytic_sol_func(t, x)
    sum([(8 / (k^3 * pi^3)) * sin(k * pi * x) * cos(1 * k * pi * t) for k in 1:2:50000])
end

u_predict = reshape([first(phi([t, x], res.u)) for t in ts for x in xs],
    (length(ts), length(xs)))
u_real = reshape([analytic_sol_func(t, x) for t in ts for x in xs],
    (length(ts), length(xs)))

diff_u = abs.(u_predict .- u_real)
p1 = plot(ts, xs, u_real, linetype = :contourf, title = "analytic");
p2 = plot(ts, xs, u_predict, linetype = :contourf, title = "predict");
p3 = plot(ts, xs, diff_u, linetype = :contourf, title = "error");
plot(p1, p2, p3)

attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
